### Transformer 理论模型

这个笔记本存储了关于 Transformer 的一堆分析，例如估算 FLOPs、参数数量、峰值内存占用、检查点大小等。

In [2]:
from collections import OrderedDict

In [3]:
# config_args = {
#     'gpt2':         dict(n_layer=12, n_head=12, n_embd=768),  # 124M 参数
#     'gpt2-medium':  dict(n_layer=24, n_head=16, n_embd=1024), # 350M 参数
#     'gpt2-large':   dict(n_layer=36, n_head=20, n_embd=1280), # 774M 参数
#     'gpt2-xl':      dict(n_layer=48, n_head=25, n_embd=1600), # 1558M 参数
# }[model_type]

block_size = 1024
vocab_size = 50257
n_layer = 12
n_head = 12
n_embd = 768
bias = False
assert not bias, "这个笔记本假设 bias=False，只是为了简化计算"

In [4]:
def params():
    """估算模型中的参数数量"""
    out = OrderedDict()

    # 词嵌入和位置嵌入
    out['emebedding/position'] = n_embd * block_size
    out['embedding/token'] = n_embd * vocab_size
    out['embedding'] = out['emebedding/position'] + out['embedding/token']

    # 注意力模块
    out['attention/ln'] = n_embd # 注意，我们的层归一化中 bias=False
    out['attention/kqv'] = n_embd * 3*n_embd
    out['attention/proj'] = n_embd**2
    out['attention'] = out['attention/ln'] + out['attention/kqv'] + out['attention/proj']

    # MLP 模块
    ffw_size = 4*n_embd # 前馈网络大小
    out['mlp/ln'] = n_embd
    out['mlp/ffw'] = n_embd * ffw_size
    out['mlp/proj'] = ffw_size * n_embd
    out['mlp'] = out['mlp/ln'] + out['mlp/ffw'] + out['mlp/proj']
    
    # Transformer 和其他部分
    out['block'] = out['attention'] + out['mlp']
    out['transformer'] = n_layer * out['block']
    out['ln_f'] = n_embd # 最终的层归一化
    out['dense'] = 0 # 因为参数共享，这个层使用嵌入层的权重，所以为 0

    # 总数
    out['total'] = out['embedding'] + out['transformer'] + out['ln_f'] + out['dense']

    return out

# 将我们的参数计数与 PyTorch 报告的进行比较
p = params()
params_total = p['total']
print(f"we see: {params_total}, expected: {124337664}, match: {params_total == 124337664}")
# 创建表头
print(f"{'name':20s} {'params':10s} {'ratio (%)':10s}")
for k,v in p.items():
    print(f"{k:20s} {v:10d} {v/params_total*100:10.4f}")
    

we see: 124337664, expected: 124337664, match: True
name                 params     ratio (%) 
emebedding/position      786432     0.6325
embedding/token        38597376    31.0424
embedding              39383808    31.6749
attention/ln                768     0.0006
attention/kqv           1769472     1.4231
attention/proj           589824     0.4744
attention               2360064     1.8981
mlp/ln                      768     0.0006
mlp/ffw                 2359296     1.8975
mlp/proj                2359296     1.8975
mlp                     4719360     3.7956
block                   7079424     5.6937
transformer            84953088    68.3245
ln_f                        768     0.0006
dense                         0     0.0000
total                 124337664   100.0000


In [5]:
# 我们现在可以计算每个检查点的大小
# 参数以 fp32 存储，AdamW 优化器为每个参数额外有 2 个缓冲区用于统计
params_bytes = params_total*4
params_and_buffers_bytes = params_bytes + 2*params_bytes
print(f"est checkpoint size: {params_and_buffers_bytes/1e9:.2f} GB")
measured_bytes = 1542470366 # 从 wc -c ckpt.pt 测量得到
print(f"measured with wc -c ckpt.pt: {measured_bytes}")
print(f"fluff ratio: {measured_bytes/params_and_buffers_bytes*100:.2f}%")

est checkpoint size: 1.49 GB
measured with wc -c ckpt.pt: 1542470366
fluff ratio: 103.38%


我们还可以估算 GPU 内存中仅由权重和 AdamW 优化器缓冲区占用的比例

In [5]:
gpu_memory = 40e9 # 40 GB A100 GPU，大约
print(f"memory ratio taken up just for parameters: {params_and_buffers_bytes / gpu_memory * 100:.2f}%")

memory ratio taken up just for parameters: 3.73%


也就是说，对于这个小型模型，内存占用并不多，大部分内存用于激活值（前向和反向传播）。当然，对于越来越大的模型，这一比例会显著变化。

让我们估算单次前向传播的 FLOPs。

In [6]:
def flops():
    # 我们只计算权重的 FLOPs，其他层（LayerNorm、Softmax 等）的计算量几乎可以忽略
    # 我们计算实际的 FLOPs，而不是 MACs，因此到处都有 2*
    # 对于任何矩阵乘法 A (BxC) @ B (CxD) -> (BxD)，FLOPs 为 2*B*C*D

    out = OrderedDict()
    head_size = n_embd // n_head

    # 注意力模块
    # 1) 投影到 key、query、value
    out['attention/kqv'] = 2 * block_size * (n_embd * 3*n_embd)
    # 2) 计算注意力分数
    out['attention/scores'] = 2 * block_size * block_size * n_embd
    # 3) 值的加权聚合 (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)
    out['attention/reduce'] = 2 * n_head * (block_size * block_size * head_size)
    # 4) 最后的线性投影
    out['attention/proj'] = 2 * block_size * (n_embd * n_embd)
    out['attention'] = sum(out['attention/'+k] for k in ['kqv', 'scores', 'reduce', 'proj'])

    # MLP 模块
    ffw_size = 4*n_embd # 前馈网络大小
    out['mlp/ffw1'] = 2 * block_size * (n_embd * ffw_size)
    out['mlp/ffw2'] = 2 * block_size * (ffw_size * n_embd)
    out['mlp'] = out['mlp/ffw1'] + out['mlp/ffw2']

    # Transformer 和其他部分
    out['block'] = out['attention'] + out['mlp']
    out['transformer'] = n_layer * out['block']
    out['dense'] = 2 * block_size * (n_embd * vocab_size)

    # 前向、反向、总数
    out['forward_total'] = out['transformer'] + out['dense']
    out['backward_total'] = 2 * out['forward_total'] # 使用常见的反向 = 2*前向估计
    out['total'] = out['forward_total'] + out['backward_total']

    return out
    
# 将我们的参数计数与 PyTorch 报告的进行比较
f = flops()
flops_total = f['forward_total']
print(f"{'name':20s} {'flops':14s} {'ratio (%)':10s}")
for k,v in f.items():
    print(f"{k:20s} {v:14d} {v/flops_total*100:10.4f}")
    

name                 flops          ratio (%) 
attention/kqv            3623878656     1.2426
attention/scores         1610612736     0.5522
attention/reduce         1610612736     0.5522
attention/proj           1207959552     0.4142
attention                8053063680     2.7612
mlp/ffw1                 4831838208     1.6567
mlp/ffw2                 4831838208     1.6567
mlp                      9663676416     3.3135
block                   17716740096     6.0747
transformer            212600881152    72.8963
dense                   79047426048    27.1037
forward_total          291648307200   100.0000
backward_total         583296614400   200.0000
total                  874944921600   300.0000


In [7]:
# 这里是从 PaLM 论文中复制的一个估算公式
# 这个公式常用于计算 MFU（模型 FLOPs 利用率）
def palm_flops():
    """根据 PaLM 论文公式估算模型 FLOPs"""
    # 非嵌入模型参数。注意，我们不减去嵌入/位置参数，因为它们是共享的，并在最后一层使用
    N = params()['total'] - params()['emebedding/position']
    L, H, Q, T = n_layer, n_head, n_embd//n_head, block_size
    mf_per_token = 6*N + 12*L*H*Q*T
    mf = mf_per_token * block_size
    return mf

print(f"palm_flops: {palm_flops():d}, flops: {flops()['total']:d}, ratio: {palm_flops()/flops()['total']:.4f}")

palm_flops: 875062886400, flops: 874944921600, ratio: 1.0001


它们非常相似，这让我对 flops() 函数中的数学计算有了一些信心。现在，A100 在张量核心上被引用为 312TFLOPS bfloat16。那么我们的模型 FLOPs 利用率（MFU）是多少？我用 batch_size 为 20 和 grad_accum 为 5 训练了上面的模型，在单个 A100 GPU 上大约运行 755ms。我们得到：

In [8]:
# 这里是我们目前大致测量的结果
batch_size = 20 * 5 # 5 是梯度累积，所以总批大小为 100
measured_time = 0.755 # 每次迭代的秒数
measured_throughput = batch_size / measured_time
flops_achieved = f['total'] * measured_throughput

# A100 被引用为在张量核心上运行 bfloat16 时为 312 TFLOPS
a100_flops_promised = 312e12

# 我们使用的 A100 部分占比：
print(f"fraction of A100 used: {flops_achieved / a100_flops_promised * 100:.2f}%")

fraction of A100 used: 37.14%


作为参考，我们希望达到 50% 以上，不仅仅对于单个 GPU，而是对于整个 DDP 运行。所以我们还有一些工作要做，但至少我们距离这个 GPU 可实现的性能只差大约 2 倍。

In [9]:
# 最后让我们检查 6ND 近似值作为训练总成本的 FLOPs
model_size = params()['total'] # 这是参数数量，N
tokens_num = 300e9 # 3000 亿个 token，这是数据集大小，D
a100_flops = 312e12 # 312 TFLOPS
assumed_mfu = 0.3 # 假设这个模型的 FLOPs 利用率（取上面的 37% 并加上一些 DDP 开销）
flops_throughput = a100_flops * 8 * assumed_mfu # 假设一个 8xA100 节点，30% 利用率
flops_needed = 6 * model_size * tokens_num # 6ND
time_needed_s = flops_needed / flops_throughput # 以秒为单位
print(f"time needed to train the model: {time_needed_s/3600/24:.2f} days")

time needed to train the model: 3.46 days


这个估算一点也不差。我训练了这个模型，它大约在 4 天内收敛。顺便说一句，关于 6ND 的来源和一些直观理解，我推荐 [Dzmitry 的帖子](https://medium.com/@dzmitrybahdanau/the-flops-calculus-of-language-model-training-3b19c1f025e4)。

现在，FLOPs 只是一种约束，我们还需要密切关注内存带宽。TODO 稍后估算我们模型的 LOAD/STORE 成本。